In [6]:
from torch_geometric.datasets import QM9
from torch_geometric.loader import DataLoader
from torch.utils.data import DataLoader
import torch
from typing import Counter

In [26]:
class TransMolecule(object):

    def __init__(self, molecule_num=5):
        self.molecule_num = molecule_num

    def __call__(self, sample):
        one_hot = sample.x[:, 0:self.molecule_num]
        molecule_counts = torch.sum(one_hot, dim=0)

        return molecule_counts

In [28]:
data = QM9(root='./practice_data', transform=TransMolecule(molecule_num=5))

"""
each batch is considered a hugh graph with many nodes and edges,
in EGNN, they introduce the concept of l2 distance between nodes, 
yet I am not including this (probably not) for now. 


"""
dataloader = DataLoader(data, batch_size=32, shuffle=False) 
for i , x in enumerate(dataloader):
    print(x)
    if i > 20: 
        break

tensor([[ 4.,  1.,  0.,  0.,  0.],
        [ 3.,  0.,  1.,  0.,  0.],
        [ 2.,  0.,  0.,  1.,  0.],
        [ 2.,  2.,  0.,  0.,  0.],
        [ 1.,  1.,  1.,  0.,  0.],
        [ 2.,  1.,  0.,  1.,  0.],
        [ 6.,  2.,  0.,  0.,  0.],
        [ 4.,  1.,  0.,  1.,  0.],
        [ 4.,  3.,  0.,  0.,  0.],
        [ 3.,  2.,  1.,  0.,  0.],
        [ 4.,  2.,  0.,  1.,  0.],
        [ 3.,  1.,  1.,  1.,  0.],
        [ 8.,  3.,  0.,  0.,  0.],
        [ 6.,  2.,  0.,  1.,  0.],
        [ 6.,  2.,  0.,  1.,  0.],
        [ 6.,  3.,  0.,  0.,  0.],
        [ 4.,  2.,  0.,  1.,  0.],
        [ 6.,  3.,  0.,  1.,  0.],
        [ 5.,  2.,  1.,  1.,  0.],
        [ 4.,  1.,  2.,  1.,  0.],
        [10.,  4.,  0.,  0.,  0.],
        [ 8.,  3.,  0.,  1.,  0.],
        [ 2.,  4.,  0.,  0.,  0.],
        [ 1.,  3.,  1.,  0.,  0.],
        [ 0.,  2.,  2.,  0.,  0.],
        [ 2.,  3.,  0.,  1.,  0.],
        [ 1.,  2.,  1.,  1.,  0.],
        [ 2.,  2.,  0.,  2.,  0.],
        [ 6.,  4.,  